# Titanic Passenger Survivor Analysis

## By: Tahsin Jahin Khalid

### The Challenge:

The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

### Part I: Data Preprocessing

#### Import Essential Modules

In [2]:
import numpy as np
import pandas as pd
import warnings
import os

#### Set up Notebook

In [3]:
# ignore deprecation/other warnings
warnings.filterwarnings('ignore')

#### Define Constants

In [4]:
DATA_DIR = "data"
OUTPUT_DIR = "interim_data"
TRAIN_FILE_NAME = "train.csv"
TEST_FILE_NAME = "test.csv"

#### Load Data

In [5]:
titanic_train_df = pd.read_csv(os.path.join(DATA_DIR, TRAIN_FILE_NAME))
titanic_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Preprocess Data

In [6]:
titanic_train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
print(f"Number of Records: {titanic_train_df.shape[0]}")
print(f"Number of Columns: {titanic_train_df.shape[1]}")

Number of Records: 891
Number of Columns: 12


In [8]:
col_lst = list(titanic_train_df.columns)
col_lst.remove("PassengerId")
col_lst.remove("Name")
col_lst.remove("Age")
col_lst.remove("Fare")
col_lst.remove("Ticket")
col_lst.remove("Cabin")

for col in col_lst:
    print(f"Unique Values for column '{col}': {titanic_train_df[f'{col}'].unique()}")

Unique Values for column 'Survived': [0 1]
Unique Values for column 'Pclass': [3 1 2]
Unique Values for column 'Sex': ['male' 'female']
Unique Values for column 'SibSp': [1 0 3 4 2 5 8]
Unique Values for column 'Parch': [0 1 2 5 3 4 6]
Unique Values for column 'Embarked': ['S' 'C' 'Q' nan]


Checking for `NaN` values

In [9]:
titanic_train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
titanic_train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
print(f"Number of Duplicated Records: {titanic_train_df.duplicated().sum()}")

Number of Duplicated Records: 0


The `Cabin` column seems irrelevant due to the majority of it being `NaN` values and is better deemed dropped.

In [12]:
titanic_train_df.drop(["Cabin"], axis=1, inplace=True)

In [13]:
titanic_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [14]:
titanic_train_df.isna().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

Assuming that the two missing values in `Embarked` are from `"Southampton" ("S")`

In [15]:
titanic_train_df["Embarked"] = titanic_train_df["Embarked"].fillna("S")

Filling the `NaN` values of the `"Age"` column with the median Age.

In [16]:
titanic_train_df["Age"] = titanic_train_df["Age"].fillna(titanic_train_df["Age"].median())

In [17]:
titanic_train_df.isna().sum()


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [18]:
titanic_train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked        object
dtype: object

#### Output as Pickle

In [19]:
titanic_train_df.to_pickle(os.path.join(OUTPUT_DIR, "preprocessed.pkl"))